In [ ]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install docplex
    else:
        !pip install --user docplex
import pandas as pd
from pandas import DataFrame
import numpy as np
from matplotlib import pyplot as plt
import random
import geopy.distance
from collections import defaultdict

from IPython.display import Markdown, display
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

import inspect
def retrieve_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]

stop_increase = False 

In [ ]:
def Model(b,bundle, riders, dataset, fairmodel = True):
    restos, all_orders, all_riders= dataset['restos'],dataset['orders'],dataset['riders']
    
    df_bundleDict = {b : pd.DataFrame()}
    df_bundleDict[b] = all_orders.loc[bundle[b]]
    
    #Sets
    P = [ i for i in bundle[b]] #list of pickup orders
    D = [ i+100 for i in bundle[b] ] #list of dropoff customers 
    O = [(o,d) for o,d in zip(P,D)] #Os order,dropoff (1,101),(2,102)
    R = [ i for i in riders[b] ]

    #locations
    P_loc = [[
              restos.loc[all_orders.loc[i]['restaurant']]['latitude'],
              restos.loc[all_orders.loc[i]['restaurant']]['longitude']
             ] for i in P] #resto location
    D_loc = [[x,y] for x,y in zip(df_bundleDict[b]['latitude'],df_bundleDict[b]['longitude'])] #customer location
    N = [0] + P + D 
    N_loc = [[7.08, 125.61]] + P_loc + D_loc
    loc_df = pd.DataFrame(N_loc, columns=['lat', 'long'], index=N)

    #load
    q = {i:all_orders.loc[i].demand for i in bundle[b]}
    for n in D:
        q[n] = (-1)*q[n-100]
    q[0] = 0
    
    # a and b
    timerange = {i:(all_orders.loc[i].a,all_orders.loc[i].b) for i in bundle[b]}
    for n in D:
        timerange[n] = timerange[n-100]
    timerange[0] = (0,0)
    
    # Matrix time and distance
    traveltime = { (r,i,j): 
                    bigM if j==0
                else
                    geopy.distance.geodesic(
                    (all_riders.loc[r].latitude,all_riders.loc[r].longitude), 
                    (loc_df.loc[j].lat,loc_df.loc[j].long)).km / km_hr * 60 if i == 0

                else
                    geopy.distance.geodesic(
                    (loc_df.loc[i].lat,loc_df.loc[i].long),
                    (loc_df.loc[j].lat,loc_df.loc[j].long)).km / km_hr * 60
                for i in N for j in N for r in R }
    distance = {
            (r,i,j):bigM if j==0
                    else
                    geopy.distance.geodesic(
                    (all_riders.loc[r].latitude,all_riders.loc[r].longitude), 
                    (loc_df.loc[j].lat,loc_df.loc[j].long)).km * 1000 if i == 0
                    else
                    geopy.distance.geodesic(
                    (loc_df.loc[i].lat,loc_df.loc[i].long),
                    (loc_df.loc[j].lat,loc_df.loc[j].long)).km * 1000
            for i in N for j in N for r in R }
    
    
    # Model
    from docplex.mp.model import Model
    mdl = Model('SOMV-PDPTW')
    
    ## Decision Var
    x = mdl.binary_var_cube(R, N, N, name="%s_from_%s_to_%s") 
    y = {r: mdl.binary_var(name="assign_%s" % (r)) for r in R}
    B = mdl.continuous_var_matrix(R, N,lb=0, name='arrivetime_%s_at_%s')
    Q = mdl.continuous_var_matrix(R, N, lb=0, ub=max_cap, name = 'load_of_%s_at_%s')
    
    ## Constraints
        # (2) each req must be satisfied
    mdl.add_constraints_(
        (mdl.sum(x[r,i,j] for r in R for j in N if j!=i) ==1
        for i in P),
        names='const1_'
    )

    # 3 ensures pairing conditions
    mdl.add_constraints_(
        mdl.sum(x[r,i[0],j] for r in R for j in N if j!=i) - mdl.sum(x[r,i[1],j] for r in R for j in N if j!=i) == 0 
        for i in O
        for r in R
    )

    # # 4 and 5 one out from 0 and one to go back at 0
    mdl.add_constraints_(
        mdl.sum(x[r,0,j] for j in N if j!=0) ==y[r]
        for r in R
    )

    # # 5
    mdl.add_constraints_(
        mdl.sum(x[r,d,0] for d in D if d!=0) ==y[r]
        for r in R
    )

    # 6 and 7 link x and y

    mdl.add_constraints_(
        mdl.sum(x[r,i,j] for i in N for j in N if j!=i) <= bigM * y[r]
        for r in R
    )

    # 7

    mdl.add_constraints_(
        mdl.sum(x[r,i,j] for i in N for j in N if j!=i) >= y[r]
        for r in R
    )

    # 8
    # flow conservation
    mdl.add_constraints_(
        (((mdl.sum(x[r,j,i] for j in N if i!=j)) - (mdl.sum(x[r,i,j] for j in N if i!=j)) == 0)
        for i in P+D
        for r in R),
        names = 'const8_'
    )

    # 9 
    mdl.add_constraints_(
        (B[r,j] - B[r,i] >= (st + traveltime[r,i,j])*x[r,i,j] - bigM * (1-x[r,i,j])
        for i in N
        for j in N if i!=j if j!=0
        for r in R),
        names = 'const9_'
    )


    # 10 update remaining capacity
    mdl.add_constraints_(
        (Q[r,j] - Q[r,i] >=( (q[j]*x[r,i,j]) - (bigM * (1-x[r,i,j]) ) )
        for i in N
        for j in N if i!=j
        for r in R),
        names = 'const10_'
    )

    # 11 
    mdl.add_constraints_(
        (
            B[r,i[0]] - B[r,i[1]] + traveltime[r,i[0],i[1]] * (mdl.sum(x[r,i[0],j] for j in N)) <= 0
            for i in O
            for r in R
        ), 
        names = "const11_"
    )

    # 12 get order after time a

    mdl.add_constraints_(
        (
            B[r,i] >= timerange[i][0] * (mdl.sum(x[r,i,j] for j in N))
            for i in N
            for r in R
        ),
        names = "const12_"
    )

    # 13 deliver order before time b

    mdl.add_constraints_(
        (
            B[r,i] <= timerange[i][1] * (mdl.sum(x[r,i,j] for j in N))
            for i in N
            for r in R
        ),
        names = "const13_"
    )

    # 14 p1
    mdl.add_constraints_(
        (
            max(0,q[i]) * (mdl.sum(x[r,i,j] for j in N)) <= Q[r,i] 
            for i in N
            for r in R
        ),
        names = "const14_"
    )

    #14 p2
    mdl.add_constraints_(
        (
            Q[r,i] <= min(max_cap,max_cap+q[i]) * mdl.sum(x[r,i,j] for j in N)
            for i in N
            for r in R
        ),
        names = "const14_"
    )
    
    ### Additonal 

     #   0 to dropoff not allowed
    mdl.add_constraints_(
        (x[r,0,i[1]] == 0) 
        for i in O
        for r in R
    )

    #  pickup to 0 not allowed
    mdl.add_constraints_(
        (x[r,i[0],0] == 0) 
        for i in O
        for r in R
    )

    # i!=j
    mdl.add_constraints_(
        (x[r,i,i] == 0) 
        for i in N
        for r in R
    )

    #no loops
    for r in R:
        for i in N:
            for j in N:
                mdl.add_if_then(x[r,i,j]==1, x[r,j,i]==0)

    for r in R:
        mdl.add_if_then(y[r]==0, mdl.sum(x[r,i,j] for i in N for j in N)==0)
        
        
    
    ### Fair Const
    if (fairmodel):
        for r in R:
            if all_riders.loc[r].idle <= beta_2:
                mdl.add_constraint(y[r]==0)

        for r in R:
            if all_riders.loc[r].idle >= beta_1:
                mdl.add_constraint(y[r]==1)

        mdl.add_constraints_(
            (mdl.sum(distance[r,i,j] * x[r,i,j] for i in N for j in N if j!=0) <= max_dist) 
            for r in R
        )
    
    ## Obj
    obj = mdl.sum(fcost_travel*(distance[r,i,j])*(x[r,i,j]) for i in N for j in N if j!=0 for r in R) \
            + fcost_rider * mdl.sum(y[r] for r in R)
    mdl.set_objective('min', obj)
    
    
    # Results
    mdl.set_time_limit(3600)
    solution = mdl.solve(log_output=False)
    sol_time = mdl.solve_details.time
    if sol_time > 3599:
        printmd("*** END INCREASE BASE @ {} *** \n".format(b), color='red')
        global stop_increase 
        stop_increase = True 
        return
    print(solution)
    mdl.report()
    mdl.solve_details
    solution.solve_status
    print("solve time =",sol_time)
    
    routes = {i for i in x if x[i].solution_value>0.9 }
    assigned_riders = {i for i in y if y[i].solution_value>0.9 }
    title = retrieve_name(dataset)[0] + ' bundle #' + str(b)
    print_solution(assigned_riders,routes, distance,title, model_type = fairmodel)

In [ ]:
def print_solution(assigned_riders,routes,distance,title,model_type,):
    feasible_routes = {r:[0] for r in assigned_riders}
    arcs = defaultdict(list)
    for k, * v in routes:
        arcs[k].append(v)
    for rider in arcs:
        i = 0
        a = arcs[rider]
        while a != []:
            for node in a:
                if i == node[0]:
                    j = node[1]  
                    feasible_routes[rider] += [j]
                    a.remove([i,j])
                    i = j

    route_cost={r:0 for r in assigned_riders}
    route_dist={r:0 for r in assigned_riders}
    for r,route in feasible_routes.items():
        route_cost[r] = fcost_rider + fcost_travel * sum(distance[r,i,j] for i,j in zip(route[:-1],route[1:-1]))
        route_dist[r] = sum(distance[r,i,j] for i,j in zip(route[:-1],route[1:-1]))
    
    model = 'FAIR MODEL' if model_type else 'BASE MODEL'
    
    printmd("*** {} with {} *** \n".format(model,title), color='blue')
    print(feasible_routes, route_cost,route_dist, sep='\n')
    print('')


# Main

In [ ]:
inst1_data = {
    'restos':pd.read_csv('https://docs.google.com/spreadsheets/d/1An-AyxvH0go1fONYRO-6G0F-mRnANe2F-cJ4QGwP2bM/export?gid=0&format=csv',index_col=0),
    'orders':pd.read_csv('https://docs.google.com/spreadsheets/d/1An-AyxvH0go1fONYRO-6G0F-mRnANe2F-cJ4QGwP2bM/export?gid=618856937&format=csv',index_col=0),
    'riders':pd.read_csv('https://docs.google.com/spreadsheets/d/1An-AyxvH0go1fONYRO-6G0F-mRnANe2F-cJ4QGwP2bM/export?gid=1625475323&format=csv',index_col=0),
}

inst2_data = {
    'restos':pd.read_csv('https://docs.google.com/spreadsheets/d/1An-AyxvH0go1fONYRO-6G0F-mRnANe2F-cJ4QGwP2bM/export?gid=1241448397&format=csv',index_col=0),
    'orders':pd.read_csv('https://docs.google.com/spreadsheets/d/1An-AyxvH0go1fONYRO-6G0F-mRnANe2F-cJ4QGwP2bM/export?gid=1470933557&format=csv', index_col=0),                    
    'riders':pd.read_csv('https://docs.google.com/spreadsheets/d/1An-AyxvH0go1fONYRO-6G0F-mRnANe2F-cJ4QGwP2bM/export?gid=2069000863&format=csv',index_col=0),
}




In [ ]:
bundle_i1 ={2: [1, 9, 14, 15, 18], 
            3: [2, 6, 7, 8, 17, 20], 
            4: [3, 11, 13, 16, 19], 
            1: [4, 5, 10, 12]}
riders_i1 = {2: ['rd1', 'rd7', 'rd8', 'rd20', 'rd25', 'rd26'], 
             3: ['rd2', 'rd13', 'rd14', 'rd15', 'rd16', 'rd17', 'rd18', 'rd19', 'rd23', 'rd28', 'rd29', 'rd30'],
             1: ['rd3', 'rd4', 'rd5', 'rd6', 'rd24'], 
             4: ['rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22', 'rd27']}

In [ ]:
bundle_i1_run ={
    1:[4], 
    2:[4, 5],
    3:[4, 5, 10],
    4:[4, 5, 10, 12],
    5:[1, 9, 14, 15, 18],
    6:[2, 6, 7, 8, 17, 20], 
    7:[1, 9, 14, 15, 18, 3, 11], 
    8:[1, 9, 14, 15, 18, 3, 11, 13], 
    9:[1, 9, 14, 15, 18, 3, 11, 13, 16], 
    10: [1, 9, 14, 15, 18, 3, 11, 13, 16, 19],
#   11: [1, 9, 14, 15, 18, 3, 11, 13, 16, 19, 2] fair > 1hr
}
riders_i1_run = {
    1: ['rd3', 'rd4', 'rd5', 'rd6', 'rd24'],
    2: ['rd3', 'rd4', 'rd5', 'rd6', 'rd24'],
    3: ['rd3', 'rd4', 'rd5', 'rd6', 'rd24'],
    4: ['rd3', 'rd4', 'rd5', 'rd6', 'rd24'],
    5: ['rd1', 'rd7', 'rd8', 'rd20', 'rd25', 'rd26'],
    6: ['rd2', 'rd13', 'rd14', 'rd15', 'rd16', 'rd17', 'rd18', 'rd19', 'rd23', 'rd28', 'rd29', 'rd30'],
    7: ['rd1', 'rd7', 'rd8', 'rd20', 'rd25', 'rd26','rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22', 'rd27'], 
    8: ['rd1', 'rd7', 'rd8', 'rd20', 'rd25', 'rd26','rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22', 'rd27'], 
    9: ['rd1', 'rd7', 'rd8', 'rd20', 'rd25', 'rd26','rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22', 'rd27'], 
    10:['rd1', 'rd7', 'rd8', 'rd20', 'rd25', 'rd26','rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22', 'rd27'],
#     11:['rd1', 'rd7', 'rd8', 'rd20', 'rd25', 'rd26','rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22', 'rd27','rd2', 'rd13', 'rd14', 'rd15', 'rd16', 'rd17', 'rd18', 'rd19', 'rd23', 'rd28', 'rd29', 'rd30']
}



In [ ]:
bundle_i2 ={1: [1, 2, 3, 11, 12, 19, 22],
            3: [4, 13, 14, 15, 16, 18, 20], 
            5: [5, 6, 7], 
            4: [8, 9, 10, 17, 21, 27], 
            2: [23, 24, 25, 26]}
riders_i2 = {1: ['rd1', 'rd2', 'rd3', 'rd4', 'rd5', 'rd6', 'rd7', 'rd8', 'rd23', 'rd30', 'rd33'],
             3: ['rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22'], 
             5: ['rd13', 'rd14', 'rd15', 'rd16'], 
             4: ['rd17', 'rd18', 'rd19', 'rd20', 'rd29', 'rd31', 'rd32', 'rd34'], 
             2: ['rd24', 'rd25', 'rd26', 'rd27', 'rd28']}

In [ ]:
bundle_i2_run ={
    1: [5],
    2: [5, 6],
    3: [5, 6, 7],
    4: [23, 24, 25, 26],
    5: [4, 13, 14, 15, 16],
    6: [4, 13, 14, 15, 16, 18],
    7: [4, 13, 14, 15, 16, 18, 20], 
    8: [4, 13, 14, 15, 16, 18, 20, 23], 
    9: [4, 13, 14, 15, 16, 18, 20, 23, 24], 
    10: [4, 13, 14, 15, 16, 18, 20, 23, 24, 25],
    11: [4, 13, 14, 15, 16, 18, 20, 23, 24, 25,26],
#     12: [4, 13, 14, 15, 16, 18, 20, 23, 24, 25, 26,5],fair > 1hr
#     13: [4, 13, 14, 15, 16, 18, 20, 23, 24, 25, 26,5,6],
#     14: [4, 13, 14, 15, 16, 18, 20, 23, 24, 25, 26,5,6,7]
}
riders_i2_run = {
    1: ['rd13', 'rd14', 'rd15', 'rd16'],
    2: ['rd13', 'rd14', 'rd15', 'rd16'],
    3: ['rd13', 'rd14', 'rd15', 'rd16'],
    4: ['rd24', 'rd25', 'rd26', 'rd27', 'rd28'],
    5: ['rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22'], 
    6: ['rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22'], 
    7: ['rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22'], 
    8: ['rd24', 'rd25', 'rd26', 'rd27', 'rd28','rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22'],
    9: ['rd24', 'rd25', 'rd26', 'rd27', 'rd28','rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22'], 
    10: ['rd24', 'rd25', 'rd26', 'rd27', 'rd28','rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22',],
    11: ['rd24', 'rd25', 'rd26', 'rd27', 'rd28','rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22',],
#     12: ['rd24', 'rd25', 'rd26', 'rd27', 'rd28','rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22','rd13', 'rd14', 'rd15', 'rd16'],
#     13: ['rd24', 'rd25', 'rd26', 'rd27', 'rd28','rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22','rd13', 'rd14', 'rd15', 'rd16'],
#     14: ['rd24', 'rd25', 'rd26', 'rd27', 'rd28','rd9', 'rd10', 'rd11', 'rd12', 'rd21', 'rd22','rd13', 'rd14', 'rd15', 'rd16']
}


## Running The Model


### Instance 1 Parameters:

In [ ]:
st = 2.5
max_dist = 4000
max_cap = 10
beta_1 =30 
beta_2= 5
bigM = 99999
fcost_rider = 45 
fcost_travel = 7/1000 
km_hr = 10
stop_increase = False

#### I1 Fair and Base Model: Node Increase

In [ ]:
for key in bundle_i1_run:
    Model(key, bundle_i1_run, riders_i1_run, inst1_data)
    if not stop_increase:
        Model(key, bundle_i1_run, riders_i1_run, inst1_data, fairmodel = False)

solution for: SOMV-PDPTW
objective: 52.7353
rd5_from_0_to_4=1
rd5_from_4_to_104=1
rd5_from_104_to_0=1
assign_rd5=1
arrivetime_rd5_at_4=36.833
arrivetime_rd5_at_104=45.000
load_of_rd5_at_4=4.000

* model SOMV-PDPTW solved with objective = 52.735
solve time = 0.008005857467651367


<span style='color:blue'>*** FAIR MODEL with inst1_data bundle #1 *** 
</span>

{'rd5': [0, 4, 104, 0]}
{'rd5': 52.73532307538076}
{'rd5': 1105.0461536258226}

solution for: SOMV-PDPTW
objective: 51.9312
rd6_from_0_to_4=1
rd6_from_4_to_104=1
rd6_from_104_to_0=1
assign_rd6=1
arrivetime_rd6_at_4=36.833
arrivetime_rd6_at_104=45.000
load_of_rd6_at_4=4.000

* model SOMV-PDPTW solved with objective = 51.931
solve time = 0.007575035095214844


<span style='color:blue'>*** BASE MODEL with inst1_data bundle #1 *** 
</span>

{'rd6': [0, 4, 104, 0]}
{'rd6': 51.931193947923006}
{'rd6': 990.1705639890013}

solution for: SOMV-PDPTW
objective: 72.7074
rd5_from_0_to_4=1
rd5_from_4_to_5=1
rd5_from_5_to_104=1
rd5_from_104_to_105=1
rd5_from_105_to_0=1
assign_rd5=1
arrivetime_rd5_at_4=5.000
arrivetime_rd5_at_5=15.000
arrivetime_rd5_at_104=23.326
arrivetime_rd5_at_105=55.000
load_of_rd5_at_4=4.000
load_of_rd5_at_5=5.000
load_of_rd5_at_104=1.000

* model SOMV-PDPTW solved with objective = 72.707
solve time = 0.04001498222351074


...

<span style='color:blue'>*** FAIR MODEL with inst1_data bundle #9 *** 
</span>

{'rd26': [0, 9, 13, 109, 113, 0], 'rd10': [0, 16, 14, 116, 114, 0], 'rd22': [0, 3, 103, 18, 15, 115, 118, 0], 'rd12': [0, 11, 111, 0], 'rd1': [0, 1, 101, 0]}
{'rd26': 72.8103614943644, 'rd10': 69.69823509676789, 'rd22': 71.01015108901618, 'rd12': 62.916896438681256, 'rd1': 68.6991304359401}
{'rd26': 3972.908784909199, 'rd10': 3528.3192995382697, 'rd22': 3715.7358698594526, 'rd12': 2559.5566340973223, 'rd1': 3385.5900622771587}

solution for: SOMV-PDPTW
objective: 353.389
rd1_from_0_to_1=1
rd1_from_1_to_101=1
rd1_from_101_to_0=1
rd7_from_0_to_9=1
rd7_from_9_to_14=1
rd7_from_14_to_19=1
rd7_from_19_to_109=1
rd7_from_109_to_119=1
rd7_from_114_to_0=1
rd7_from_119_to_114=1
rd26_from_0_to_15=1
rd26_from_15_to_16=1
rd26_from_18_to_115=1
rd26_from_16_to_18=1
rd26_from_115_to_116=1
rd26_from_118_to_0=1
rd26_from_116_to_118=1
rd10_from_0_to_11=1
rd10_from_11_to_111=1
rd10_from_111_to_0=1
rd22_from_0_to_13=1
rd22_from_3_to_103=1
rd22_from_13_to_3=1
rd22_from_103_to_113=1
rd22_from_113_to_0=1
assig

<span style='color:blue'>*** FAIR MODEL with inst1_data bundle #10 *** 
</span>

{'rd26': [0, 15, 16, 18, 115, 116, 118, 0], 'rd10': [0, 11, 111, 0], 'rd7': [0, 9, 14, 19, 109, 119, 114, 0], 'rd22': [0, 13, 3, 103, 113, 0], 'rd1': [0, 1, 101, 0]}
{'rd26': 71.85383286720592, 'rd10': 68.97342003667949, 'rd7': 71.81633268679434, 'rd22': 72.04644224740936, 'rd1': 68.6991304359401}
{'rd26': 3836.2618381722746, 'rd10': 3424.7742909542117, 'rd7': 3830.904669542049, 'rd22': 3863.7774639156223, 'rd1': 3385.5900622771587}



<span style='color:red'>*** END INCREASE BASE @ 11 *** 
</span>

#### I1 Fair and Base Model

In [ ]:
stop_increase = False
for key in bundle_i1:
    Model(key, bundle_i1, riders_i1, inst1_data)
    Model(key, bundle_i1, riders_i1, inst1_data, fairmodel = False)

solution for: SOMV-PDPTW
objective: 195.447
rd1_from_0_to_1=1
rd1_from_1_to_101=1
rd1_from_101_to_0=1
rd7_from_0_to_9=1
rd7_from_9_to_14=1
rd7_from_14_to_114=1
rd7_from_109_to_0=1
rd7_from_114_to_109=1
rd26_from_0_to_15=1
rd26_from_15_to_18=1
rd26_from_18_to_115=1
rd26_from_115_to_118=1
rd26_from_118_to_0=1
assign_rd1=1
assign_rd7=1
assign_rd26=1
arrivetime_rd1_at_1=33.406
arrivetime_rd1_at_101=55.000
arrivetime_rd7_at_9=10.000
arrivetime_rd7_at_14=12.500
arrivetime_rd7_at_109=34.286
arrivetime_rd7_at_114=23.109
arrivetime_rd26_at_15=10.000
arrivetime_rd26_at_18=14.624
arrivetime_rd26_at_115=19.707
arrivetime_rd26_at_118=45.000
load_of_rd1_at_1=10.000
load_of_rd7_at_9=2.000
load_of_rd7_at_14=3.000
load_of_rd7_at_114=2.000
load_of_rd26_at_15=2.000
load_of_rd26_at_18=5.000
load_of_rd26_at_115=3.000

* model SOMV-PDPTW solved with objective = 195.447
solve time = 0.2872500419616699


<span style='color:blue'>*** FAIR MODEL with inst1_data bundle #2 *** 
</span>

{'rd26': [0, 15, 18, 115, 118, 0], 'rd1': [0, 1, 101, 0], 'rd7': [0, 9, 14, 114, 109, 0]}
{'rd26': 60.658967270644574, 'rd1': 68.6991304359401, 'rd7': 66.08930708551408}
{'rd26': 2236.9953243777964, 'rd1': 3385.5900622771587, 'rd7': 3012.7581550734403}

solution for: SOMV-PDPTW
objective: 156.61
rd8_from_0_to_18=1
rd8_from_9_to_118=1
rd8_from_14_to_9=1
rd8_from_18_to_14=1
rd8_from_109_to_0=1
rd8_from_114_to_109=1
rd8_from_118_to_114=1
rd20_from_0_to_15=1
rd20_from_1_to_101=1
rd20_from_15_to_115=1
rd20_from_101_to_0=1
rd20_from_115_to_1=1
assign_rd8=1
assign_rd20=1
arrivetime_rd8_at_9=12.500
arrivetime_rd8_at_14=10.000
arrivetime_rd8_at_18=5.000
arrivetime_rd8_at_109=50.000
arrivetime_rd8_at_114=33.396
arrivetime_rd8_at_118=20.728
arrivetime_rd20_at_1=33.406
arrivetime_rd20_at_15=10.000
arrivetime_rd20_at_101=55.000
arrivetime_rd20_at_115=15.892
load_of_rd8_at_0=4.000
load_of_rd8_at_9=10.000
load_of_rd8_at_14=8.000
load_of_rd8_at_18=7.000
load_of_rd8_at_109=4.000
load_of_rd8_at_114=6.00

<span style='color:blue'>*** BASE MODEL with inst1_data bundle #2 *** 
</span>

{'rd20': [0, 15, 115, 1, 101, 0], 'rd8': [0, 18, 14, 9, 118, 114, 109, 0]}
{'rd20': 79.01491163425115, 'rd8': 77.59513129116749}
{'rd20': 4859.273090607308, 'rd8': 4656.447327309643}

solution for: SOMV-PDPTW
objective: 202.704
rd2_from_0_to_20=1
rd2_from_6_to_106=1
rd2_from_20_to_6=1
rd2_from_106_to_120=1
rd2_from_120_to_0=1
rd28_from_0_to_2=1
rd28_from_2_to_7=1
rd28_from_7_to_102=1
rd28_from_102_to_107=1
rd28_from_107_to_0=1
rd30_from_0_to_17=1
rd30_from_8_to_108=1
rd30_from_17_to_8=1
rd30_from_108_to_117=1
rd30_from_117_to_0=1
assign_rd2=1
assign_rd28=1
assign_rd30=1
arrivetime_rd2_at_6=17.500
arrivetime_rd2_at_20=15.000
arrivetime_rd2_at_106=27.018
arrivetime_rd2_at_120=45.077
arrivetime_rd23_at_20=-0.000
arrivetime_rd28_at_2=10.000
arrivetime_rd28_at_7=17.729
arrivetime_rd28_at_102=26.601
arrivetime_rd28_at_107=33.613
arrivetime_rd30_at_8=14.087
arrivetime_rd30_at_17=10.000
arrivetime_rd30_at_108=21.116
arrivetime_rd30_at_117=34.036
load_of_rd2_at_6=5.000
load_of_rd2_at_20=3.000
l

...

{'rd9': [0, 19, 119, 16, 116, 0], 'rd27': [0, 3, 103, 11, 13, 111, 113, 0]}
{'rd9': 61.622333063184016, 'rd27': 81.40245571399032}
{'rd9': 2374.6190090262885, 'rd27': 5200.350816284333}

solution for: SOMV-PDPTW
objective: 132.597
rd4_from_0_to_5=1
rd4_from_5_to_105=1
rd4_from_105_to_0=1
rd5_from_0_to_4=1
rd5_from_4_to_10=1
rd5_from_10_to_104=1
rd5_from_12_to_110=1
rd5_from_104_to_12=1
rd5_from_110_to_112=1
rd5_from_112_to_0=1
assign_rd4=1
assign_rd5=1
arrivetime_rd4_at_5=15.000
arrivetime_rd4_at_105=30.549
arrivetime_rd5_at_4=5.000
arrivetime_rd5_at_10=7.500
arrivetime_rd5_at_12=30.804
arrivetime_rd5_at_104=15.667
arrivetime_rd5_at_110=36.052
arrivetime_rd5_at_112=45.000
load_of_rd4_at_5=1.000
load_of_rd5_at_4=4.000
load_of_rd5_at_10=5.000
load_of_rd5_at_12=3.000
load_of_rd5_at_104=1.000
load_of_rd5_at_110=2.000

* model SOMV-PDPTW solved with objective = 132.597
solve time = 0.21483707427978516


<span style='color:blue'>*** FAIR MODEL with inst1_data bundle #1 *** 
</span>

{'rd5': [0, 4, 10, 104, 12, 110, 112, 0], 'rd4': [0, 5, 105, 0]}
{'rd5': 70.26092683410877, 'rd4': 62.336266060397406}
{'rd5': 3608.7038334441113, 'rd4': 2476.6094371996287}

solution for: SOMV-PDPTW
objective: 130.682
rd3_from_0_to_5=1
rd3_from_5_to_105=1
rd3_from_105_to_0=1
rd6_from_0_to_4=1
rd6_from_4_to_10=1
rd6_from_10_to_104=1
rd6_from_12_to_110=1
rd6_from_104_to_12=1
rd6_from_110_to_112=1
rd6_from_112_to_0=1
assign_rd3=1
assign_rd6=1
arrivetime_rd3_at_4=-0.000
arrivetime_rd3_at_5=15.000
arrivetime_rd3_at_105=30.549
arrivetime_rd6_at_4=5.000
arrivetime_rd6_at_10=7.500
arrivetime_rd6_at_12=23.993
arrivetime_rd6_at_104=15.667
arrivetime_rd6_at_110=29.241
arrivetime_rd6_at_112=45.000
load_of_rd3_at_0=9.000
load_of_rd3_at_5=10.000
load_of_rd3_at_105=9.000
load_of_rd6_at_4=4.000
load_of_rd6_at_10=5.000
load_of_rd6_at_12=3.000
load_of_rd6_at_104=1.000
load_of_rd6_at_110=2.000

* model SOMV-PDPTW solved with objective = 130.682
solve time = 0.706718921661377


<span style='color:blue'>*** BASE MODEL with inst1_data bundle #1 *** 
</span>

{'rd6': [0, 4, 10, 104, 12, 110, 112, 0], 'rd3': [0, 5, 105, 0]}
{'rd6': 69.45679770665103, 'rd3': 61.22485977181912}
{'rd6': 3493.82824380729, 'rd3': 2317.837110259874}



### Instance 2 Parameters:

In [ ]:
st = 2.5 
max_dist = 5500 
max_cap = 10
beta_1 =30 
beta_2= 5
bigM = 99999
fcost_rider = 45
fcost_travel = 7/1000
km_hr = 10
stop_increase = False

#### I2 Fair and Base Model: Node Increase

In [ ]:
for key in bundle_i2_run:
    Model(key, bundle_i2_run, riders_i2_run, inst2_data)
    if not stop_increase:
        Model(key, bundle_i2_run, riders_i2_run, inst2_data,fairmodel = False)    

solution for: SOMV-PDPTW
objective: 58.4609
rd13_from_0_to_5=1
rd13_from_5_to_105=1
rd13_from_105_to_0=1
assign_rd13=1
arrivetime_rd13_at_5=8.040
arrivetime_rd13_at_105=40.000
load_of_rd13_at_5=4.000

* model SOMV-PDPTW solved with objective = 58.461
solve time = 0.0054779052734375


<span style='color:blue'>*** FAIR MODEL with inst2_data bundle #1 *** 
</span>

{'rd13': [0, 5, 105, 0]}
{'rd13': 58.46085749386201}
{'rd13': 1922.9796419802874}

solution for: SOMV-PDPTW
objective: 56.2089
rd14_from_0_to_5=1
rd14_from_5_to_105=1
rd14_from_105_to_0=1
assign_rd14=1
arrivetime_rd13_at_5=-0.000
arrivetime_rd14_at_5=6.110
arrivetime_rd14_at_105=40.000
arrivetime_rd15_at_5=-0.000
arrivetime_rd16_at_5=-0.000
load_of_rd14_at_5=4.000

* model SOMV-PDPTW solved with objective = 56.209
solve time = 0.006212949752807617


<span style='color:blue'>*** BASE MODEL with inst2_data bundle #1 *** 
</span>

{'rd14': [0, 5, 105, 0]}
{'rd14': 56.20886510689117}
{'rd14': 1601.2664438415948}

solution for: SOMV-PDPTW
objective: 72.1158
rd13_from_0_to_5=1
rd13_from_5_to_105=1
rd13_from_6_to_106=1
rd13_from_105_to_6=1
rd13_from_106_to_0=1
assign_rd13=1
arrivetime_rd13_at_5=8.040
arrivetime_rd13_at_6=23.386
arrivetime_rd13_at_105=16.538
arrivetime_rd13_at_106=60.000
arrivetime_rd14_at_5=-0.000
arrivetime_rd16_at_5=-0.000
load_of_rd13_at_5=4.000
load_of_rd13_at_6=2.000

* model SOMV-PDPTW solved with objective = 72.116
solve time = 0.025515079498291016


...

{'rd21': [0, 4, 104, 0], 'rd26': [0, 25, 24, 23, 125, 123, 124, 0], 'rd10': [0, 14, 18, 16, 114, 118, 116, 0], 'rd25': [0, 15, 115, 0], 'rd9': [0, 13, 20, 113, 120, 0]}
{'rd21': 58.60966188168577, 'rd26': 75.19787800482148, 'rd10': 76.30006697910079, 'rd25': 75.11012592790416, 'rd9': 73.8006618435817}
{'rd21': 1944.2374116693954, 'rd26': 4313.982572117353, 'rd10': 4471.438139871542, 'rd25': 4301.446561129167, 'rd9': 4114.380263368813}

solution for: SOMV-PDPTW
objective: 427.366
rd24_from_0_to_23=-0
rd24_from_0_to_26=1
rd24_from_13_to_20=-0
rd24_from_13_to_104=0
rd24_from_13_to_113=-0
rd24_from_14_to_104=-0
rd24_from_15_to_120=-0
rd24_from_16_to_18=0
rd24_from_16_to_104=-0
rd24_from_18_to_16=0
rd24_from_20_to_13=0
rd24_from_20_to_16=-0
rd24_from_23_to_26=-0
rd24_from_23_to_120=0
rd24_from_23_to_123=1
rd24_from_26_to_23=1
rd24_from_26_to_116=-0
rd24_from_26_to_118=0
rd24_from_26_to_123=-0
rd24_from_104_to_13=-0
rd24_from_104_to_20=0
rd24_from_113_to_14=-0
rd24_from_113_to_26=0
rd24_from

<span style='color:blue'>*** FAIR MODEL with inst2_data bundle #11 *** 
</span>

{'rd26': [0, 24, 25, 125, 124, 0], 'rd21': [0, 4, 104, 0], 'rd10': [0, 14, 16, 18, 114, 118, 116, 0], 'rd25': [0, 15, 115, 0], 'rd24': [0, 26, 23, 123, 126, 0], 'rd9': [0, 13, 20, 113, 120, 0]}
{'rd26': 72.63494050750556, 'rd21': 58.60966188168577, 'rd10': 76.30006697910079, 'rd25': 75.11012592790416, 'rd24': 70.9106632801407, 'rd9': 73.8006618435817}
{'rd26': 3947.8486439293656, 'rd21': 1944.2374116693954, 'rd10': 4471.438139871542, 'rd25': 4301.446561129167, 'rd24': 3701.5233257343857, 'rd9': 4114.380263368813}



<span style='color:red'>*** END INCREASE BASE @ 12 *** 
</span>

<span style='color:red'>*** END INCREASE BASE @ 13 *** 
</span>

#### I2 Fair and Base Model

In [ ]:
stop_increase = False
for key in bundle_i2:
    Model(key, bundle_i2, riders_i2, inst2_data)
    Model(key, bundle_i2, riders_i2, inst2_data,fairmodel = False)  

solution for: SOMV-PDPTW
objective: 275.032
rd1_from_0_to_1=1
rd1_from_1_to_101=1
rd1_from_2_to_102=0
rd1_from_2_to_112=-0
rd1_from_12_to_112=1
rd1_from_101_to_12=1
rd1_from_102_to_112=0
rd1_from_112_to_0=1
rd2_from_0_to_1=-0
rd2_from_1_to_101=-0
rd2_from_12_to_112=-0
rd2_from_101_to_12=-0
rd2_from_112_to_0=-0
rd5_from_0_to_2=1
rd5_from_2_to_102=1
rd5_from_102_to_0=1
rd8_from_1_to_101=0
rd8_from_1_to_102=-0
rd8_from_12_to_101=-0
rd8_from_12_to_112=0
rd8_from_101_to_12=0
rd8_from_102_to_112=-0
rd23_from_0_to_1=0
rd23_from_0_to_22=1
rd23_from_11_to_111=1
rd23_from_22_to_11=1
rd23_from_111_to_122=1
rd23_from_112_to_1=-0
rd23_from_112_to_22=0
rd23_from_122_to_0=1
rd30_from_0_to_19=1
rd30_from_3_to_103=1
rd30_from_19_to_3=1
rd30_from_103_to_119=1
rd30_from_119_to_0=1
assign_rd1=1
assign_rd2=-0
assign_rd5=1
assign_rd23=1
assign_rd30=1
arrivetime_rd1_at_1=4.465
arrivetime_rd1_at_12=25.000
arrivetime_rd1_at_101=9.824
arrivetime_rd1_at_112=36.819
arrivetime_rd2_at_12=-0.000
arrivetime_rd2_at_11

<span style='color:blue'>*** FAIR MODEL with inst2_data bundle #1 *** 
</span>

{'rd23': [0, 22, 11, 111, 122, 0], 'rd1': [0, 1, 101, 12, 112, 0], 'rd5': [0, 2, 102, 0], 'rd30': [0, 19, 3, 103, 119, 0]}
{'rd23': 71.66378084005846, 'rd1': 64.17978649664852, 'rd5': 59.80291126349414, 'rd30': 79.38583907626787}
{'rd23': 3809.1115485797804, 'rd1': 2739.9694995212167, 'rd5': 2114.7016090705915, 'rd30': 4912.262725181125}

solution for: SOMV-PDPTW
objective: 239.94
rd5_from_0_to_2=1
rd5_from_2_to_102=1
rd5_from_11_to_111=1
rd5_from_22_to_11=1
rd5_from_102_to_22=1
rd5_from_111_to_122=1
rd5_from_122_to_0=1
rd6_from_0_to_19=1
rd6_from_3_to_103=1
rd6_from_19_to_3=1
rd6_from_103_to_119=1
rd6_from_119_to_0=1
rd23_from_0_to_1=1
rd23_from_1_to_101=1
rd23_from_12_to_112=1
rd23_from_101_to_12=1
rd23_from_112_to_0=1
assign_rd5=1
assign_rd6=1
assign_rd23=1
arrivetime_rd1_at_1=-0.000
arrivetime_rd1_at_19=-0.000
arrivetime_rd1_at_101=-0.000
arrivetime_rd2_at_1=-0.000
arrivetime_rd3_at_1=-0.000
arrivetime_rd3_at_101=-0.000
arrivetime_rd4_at_2=-0.000
arrivetime_rd5_at_2=5.615
arrivetim

<span style='color:blue'>*** BASE MODEL with inst2_data bundle #1 *** 
</span>

{'rd23': [0, 1, 101, 12, 112, 0], 'rd5': [0, 2, 102, 22, 11, 111, 122, 0], 'rd6': [0, 19, 3, 103, 119, 0]}
{'rd23': 62.927538162362126, 'rd5': 99.90257291532379, 'rd6': 77.11037203003134}
{'rd23': 2561.0768803374463, 'rd5': 7843.224702189113, 'rd6': 4587.196004290192}

solution for: SOMV-PDPTW
objective: 286.462
rd9_from_0_to_13=1
rd9_from_13_to_20=1
rd9_from_20_to_113=1
rd9_from_113_to_120=1
rd9_from_120_to_0=1
rd10_from_0_to_14=1
rd10_from_14_to_16=1
rd10_from_16_to_18=1
rd10_from_18_to_114=1
rd10_from_114_to_118=1
rd10_from_116_to_0=1
rd10_from_118_to_116=1
rd21_from_0_to_4=1
rd21_from_4_to_104=1
rd21_from_104_to_0=1
rd22_from_0_to_15=1
rd22_from_15_to_115=1
rd22_from_115_to_0=1
assign_rd9=1
assign_rd10=1
assign_rd21=1
assign_rd22=1
arrivetime_rd9_at_13=5.000
arrivetime_rd9_at_16=-0.000
arrivetime_rd9_at_20=7.500
arrivetime_rd9_at_113=16.323
arrivetime_rd9_at_120=45.000
arrivetime_rd10_at_14=25.000
arrivetime_rd10_at_16=30.429
arrivetime_rd10_at_18=32.929
arrivetime_rd10_at_114=38.8

...

{'rd32': [0, 21, 9, 109, 121, 0], 'rd17': [0, 8, 108, 10, 17, 27, 117, 110, 127, 0]}
{'rd32': 76.55403087148274, 'rd17': 90.59738249282121}
{'rd32': 4507.718695926105, 'rd17': 6513.911784688744}

solution for: SOMV-PDPTW
objective: 145.68
rd24_from_24_to_25=0
rd24_from_25_to_24=0
rd24_from_125_to_24=0
rd25_from_0_to_24=-0
rd25_from_0_to_25=1
rd25_from_0_to_26=0
rd25_from_24_to_25=-0
rd25_from_24_to_124=0
rd25_from_24_to_125=1
rd25_from_25_to_24=1
rd25_from_25_to_125=-0
rd25_from_26_to_125=0
rd25_from_124_to_0=1
rd25_from_125_to_124=1
rd26_from_0_to_23=0
rd26_from_0_to_25=-0
rd26_from_0_to_26=1
rd26_from_23_to_123=1
rd26_from_25_to_23=-0
rd26_from_25_to_123=0
rd26_from_26_to_23=1
rd26_from_26_to_123=-0
rd26_from_123_to_126=1
rd26_from_126_to_0=1
assign_rd25=1
assign_rd26=1
arrivetime_rd24_at_24=0.000
arrivetime_rd24_at_25=0.000
arrivetime_rd24_at_124=0.000
arrivetime_rd24_at_125=0.000
arrivetime_rd25_at_23=-0.000
arrivetime_rd25_at_24=30.000
arrivetime_rd25_at_25=27.500
arrivetime_rd25_

<span style='color:blue'>*** FAIR MODEL with inst2_data bundle #2 *** 
</span>

{'rd26': [0, 26, 23, 123, 126, 0], 'rd25': [0, 25, 24, 125, 124, 0]}
{'rd26': 72.63332355049076, 'rd25': 73.04653525240462}
{'rd26': 3947.6176500701085, 'rd25': 4006.6478932006607}

solution for: SOMV-PDPTW
objective: 143.546
rd24_from_0_to_26=1
rd24_from_23_to_123=1
rd24_from_26_to_23=1
rd24_from_123_to_126=1
rd24_from_126_to_0=1
rd25_from_24_to_25=-0
rd25_from_25_to_24=-0
rd25_from_123_to_124=0
rd25_from_124_to_123=0
rd26_from_0_to_23=0
rd26_from_0_to_24=1
rd26_from_0_to_25=0
rd26_from_23_to_125=0
rd26_from_24_to_23=0
rd26_from_24_to_25=1
rd26_from_25_to_24=0
rd26_from_25_to_125=1
rd26_from_124_to_0=1
rd26_from_125_to_124=1
rd27_from_24_to_25=0
rd27_from_25_to_24=0
rd27_from_123_to_124=-0
rd27_from_124_to_123=-0
rd28_from_23_to_125=-0
rd28_from_24_to_23=-0
rd28_from_24_to_125=0
assign_rd24=1
assign_rd26=1
arrivetime_rd24_at_23=7.500
arrivetime_rd24_at_26=5.000
arrivetime_rd24_at_123=21.013
arrivetime_rd24_at_126=45.000
arrivetime_rd25_at_23=-0.000
arrivetime_rd25_at_123=-0.000
arrive

<span style='color:blue'>*** BASE MODEL with inst2_data bundle #2 *** 
</span>

{'rd24': [0, 26, 23, 123, 126, 0], 'rd26': [0, 24, 25, 125, 124, 0]}
{'rd24': 70.9106632801407, 'rd26': 72.63494050750556}
{'rd24': 3701.5233257343857, 'rd26': 3947.8486439293656}

